In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core

!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

# Register dataset to detectron2

In [ ]:
from pathlib import Path

In [ ]:

is_resume_training = True

# Path to your pretrained model weights 
PRETRAINED_PATH = Path("/kaggle/input/output/output (1)/model_final.pth")

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
%%capture
# detectron2
from detectron2.utils.memory import retry_if_cuda_oom
from detectron2.utils.logger import setup_logger
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
import detectron2.data.transforms as T
from detectron2.data import detection_utils as utils
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader, DatasetMapper
from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm  # progress bar
import matplotlib.pyplot as plt
import json
import cv2
import copy
from typing import Optional
import seaborn as sns

!pip install -q pycocotools
from pycocotools.coco import COCO
from PIL import Image
import random
from pathlib import Path
%matplotlib inline
sns.set_theme(style='darkgrid', palette='deep', font='sans-serif', font_scale=1)

from IPython.display import FileLink
# torch
import torch
import gc
import warnings
# Ignore "future" warnings and Data-Frame-Slicing warnings.
warnings.filterwarnings('ignore')

setup_logger()

In [ ]:
OUTPUT_DIR = Path("./output")

In [ ]:
def cv2_imshow(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.figure(), plt.imshow(im), plt.axis('off');

In [ ]:
import os
import cv2
import json
import numpy as np
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

def get_balloon_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_export.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    category_mapping = {
        'Black_Scruf': 0,
        'Common_Scab': 1,
    }

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}
        #print(idx)
        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        annos = v["regions"]
        #print(filename)
        objs = []
        for anno in annos:
            shape_attributes = anno["shape_attributes"]
            region_attributes = anno["region_attributes"]
            #print(filename)
            px = shape_attributes["all_points_x"]
            py = shape_attributes["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]
            #print(region_attributes)
            category_name = region_attributes["categories"]
            #print(category_name)
            category_id = category_mapping.get(category_name, 0)

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": category_id,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts




In [ ]:
for d in ["train"]:
    DatasetCatalog.register("Potato_" + d, lambda d=d: get_balloon_dicts("/kaggle/input/potato-seg/pot/pot/" + d))
    MetadataCatalog.get("Potato_" + d).set(thing_classes=["Common_Scab","Black_Scruf"])

balloon_metadata = MetadataCatalog.get("Potato_train")

print(balloon_metadata)

In [ ]:
balloon_metadata = MetadataCatalog.get("Potato_train")

In [ ]:
print(balloon_metadata)

In [ ]:
import torch

print(torch.cuda.is_available())


In [ ]:
dataset_train = DatasetCatalog.get("Potato_train")

# To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set

In [ ]:

import matplotlib.pyplot as plt
dataset_dicts = get_balloon_dicts("/kaggle/input/potato-seg/pot/pot/train")
for d in dataset_dicts[:1]:
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=balloon_metadata)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:

from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.MODEL.DEVICE = "cuda"

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("Potato_valid")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 1
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
#cfg.MODEL.WEIGHTS = str(PRETRAINED_PATH)
    
#mask_rcnn_X_101_32x8d_FPN_3x.yaml 
#mask_rcnn_R_50_FPN_1x.yaml mask_rcnn_R_101_FPN_3x.yaml

cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.001  
cfg.SOLVER.MAX_ITER = 2000   
cfg.SOLVER.STEPS = []        
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # Number of class

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()


In [ ]:

    # Load metrics
    metrics_df = pd.read_json(
        OUTPUT_DIR/"metrics.json", orient="records", lines=True
    )
    mdf = metrics_df.sort_values("iteration")
    #.tail(3)
    print(mdf.tail(3).T)

    # Plot loss
    fig, ax = plt.subplots()

    mdf1 = mdf[~mdf["total_loss"].isna()]
    ax.plot(mdf1["iteration"], mdf1["total_loss"], c="C0", label="train")

    if "validation_loss" in mdf.columns:
        mdf2 = mdf[~mdf["validation_loss"].isna()]
        ax.plot(mdf2["iteration"], mdf2["validation_loss"],
                c="C1", label="validation")

    ax.legend()
    ax.set_title("Loss curve")
    plt.show()

    # Plot Accuracy
    fig, ax = plt.subplots()

    mdf1 = mdf[~mdf["mask_rcnn/accuracy"].isna()]
    ax.plot(mdf1["iteration"], mdf1["mask_rcnn/accuracy"],
            c="C0", label="train")

    ax.legend()
    ax.set_title("Accuracy curve")
    plt.show()

    # Plot Bounding Box regressor loss
    fig, ax = plt.subplots()

    mdf1 = mdf[~mdf["loss_box_reg"].isna()]
    ax.plot(mdf1["iteration"], mdf1["loss_box_reg"], c="C0", label="train")

    ax.legend()
    ax.set_title("loss_box_reg")
    plt.show()

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold

predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("Potato_valid", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "Potato_valid")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

In [ ]:
from detectron2.utils.visualizer import ColorMode
fig, ax = plt.subplots(4, 2, figsize=(10, 10))
indices = [ax[0][0], ax[1][0], ax[0][1], ax[1][1],
           ax[2][0], ax[3][0], ax[2][1], ax[3][1]]
i=0
dataset_dicts = get_balloon_dicts("/kaggle/input/potato-seg/pot/pot/train/")
for d in dataset_dicts[50:58]:
    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=balloon_metadata, 
                   
                   instance_mode=ColorMode.IMAGE_BW,
                   # remove the colors of unsegmented pixels. This option is only available for segmentation models
                    # You can adjust this value to set the font size
    )
    visualizer = v.draw_instance_predictions(
        outputs["instances"].to("cpu")
    )
    display_img = visualizer.get_image()[:, :, ::-1]
    indices[i].grid(False)
    indices[i].imshow(display_img)
    i=i+1


In [ ]:
from detectron2.utils.visualizer import ColorMode, Visualizer
import matplotlib.pyplot as plt
import cv2
import random

fig, ax = plt.subplots(25, 2, figsize=(50, 50))
indices = [ax[i // 2, i % 2] for i in range(50)]  # Flatten the 2D array
i = 0
dataset_dicts = get_balloon_dicts("/kaggle/input/potato-seg/pot/pot/train")

for d in dataset_dicts[:50]:  # Loop through the first 50 images
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    
    v = Visualizer(im[:, :, ::-1],
                   metadata=balloon_metadata,
                   instance_mode=ColorMode.IMAGE_BW,
    )
    
    visualizer = v.draw_instance_predictions(
        outputs["instances"].to("cpu")
    )
    
    display_img = visualizer.get_image()[:, :, ::-1]
    indices[i].grid(False)
    indices[i].imshow(display_img)
    i += 1



In [ ]:
custom_data_list = get_balloon_dicts("/kaggle/input/potato-seg/pot/pot/valid")

In [ ]:
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultPredictor
from detectron2.structures import BoxMode
import torch
import cv2
import numpy as np
# cfg now already contains everything we've set previously. We changed it a little bit for inference:

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold

predictor = DefaultPredictor(cfg)

# Assume custom_data_list is a list of dictionaries with 'file_name', 'annotations', etc.
dice_scores = []

for data in custom_data_list:
    # Load the image
    file_path = data["file_name"]
    image = cv2.imread(file_path)

    # Use the predictor to get predictions
    outputs = predictor(image)

    # Extract predicted bounding boxes
    pred_boxes = outputs["instances"].pred_boxes.tensor.cpu().numpy()

    # Extract ground truth bounding boxes
    gt_box = data["annotations"][0]["bbox"]  # Adjust this based on your annotation structure
    gt_box_detectron2 = BoxMode.convert(gt_box, BoxMode.XYXY_ABS, BoxMode.XYXY_ABS)
    # Check if there are predictions
    if pred_boxes.size == 0:
        # No predictions for this image, handle accordingly
        continue
    # Calculate TP, FP, and FN
    x_min = max(gt_box_detectron2[0], pred_boxes[:, 0].min())
    y_min = max(gt_box_detectron2[1], pred_boxes[:, 1].min())
    x_max = min(gt_box_detectron2[2], pred_boxes[:, 2].max())
    y_max = min(gt_box_detectron2[3], pred_boxes[:, 3].max())

    intersection_area = max(0, x_max - x_min) * max(0, y_max - y_min)
    gt_box_area = (gt_box_detectron2[2] - gt_box_detectron2[0]) * (gt_box_detectron2[3] - gt_box_detectron2[1])
    pred_box_area = np.sum((pred_boxes[:, 2] - pred_boxes[:, 0]) * (pred_boxes[:, 3] - pred_boxes[:, 1]))

    TP = intersection_area
    FP = pred_box_area - TP
    FN = gt_box_area - TP

    # Calculate Dice score
    dice_score = (2 * TP) / (2 * TP + FP + FN)

    # Append Dice score to the list
    dice_scores.append(dice_score)

# Calculate mean Dice score
mean_dice_score = np.mean(dice_scores)
print("Mean Dice Score:", mean_dice_score)
